In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [74]:
def dissocierList(L,j):
    
    res = []
    
    for i in range(len(L)):
        res.append(L[i][j])
    
    return res

In [75]:
def addProcess_2S(tmax=300, lama=30, l_lamd=[30], l_nbPers=[0]):
    
    lTmps = [0]
    t = 0.0  # temps du derniere evenement
    assert len(l_lamd) == len(l_nbPers)
    nbService = len(l_lamd)
    # None represente une valeur a definir dans la boucle
    dta = None  # delta inter arrivee
    dtd = nbService * [None]  # deltas inter departs
    cpt = l_nbPers 
    cpt_T = [np.copy(cpt)]  # liste qui accumule les etats des files d'attente a chaque evenement 
        
    while True:

        # dtdMin = np.argmin(dtd)
        # cptMin = np.argmin(cpt)
        
        if dta is None:
            # la prochaine arrivee n'est pas definie, on la redefinie 
            dta = np.random.exponential(lama)
            
        for i in range(nbService):
            if dtd[i] is None and cpt[i] > 0:
                # le prochain depart n'est definie pour cette file, on le redefini
                dtd[i] = np.random.exponential(l_lamd[i])
        
        # on recherche le delta le plus proche dans le temps
        dtdMin = np.inf
        imin = None
        for i in range(nbService):
            if dtd[i] is not None and dtdMin > dtd[i]:
                dtdMin = dtd[i]
                imin = i
            
        if imin is not None:
            # cas ou l'arrivee est en concurence avec les departs
            if dtd[imin] > dta:
                #l'arrivee se produit avant les desparts
                cptMin = np.inf
                cpt_i = None
                t += dta
                for i in range(nbService):
                    if dtd[i] is not None:
                        dtd[i] -= dta
                    if cpt[i] < cptMin:
                        cptMin = cpt[i]
                        cpt_i = i   
                cpt[cpt_i] += 1
                dta = None          
            else:
                # le depart le plus proche se produit avant la prochaine arrivee
                t += dtd[imin]
                dta -= dtd[imin]
                for i in range(nbService):
                    if dtd[i] is not None:
                        dtd[i] -= dtd[imin]
                dtd[imin] = None
                cpt[imin] -= 1
        
        else:
            #cas ou l'arrivee est la seul possibilitee
            t += dta
            k = np.random.randint(nbService)  # choix d'une file au hasard
            cpt[k] += 1
            dta = None  
            
        #verification de si on ne depasse pas les bornes de la simulation
        if t > tmax:
            break

        lTmps.append(t)
        cpt_T.append(np.copy(cpt))
        
    return lTmps,cpt_T  

In [166]:
def g_Max(lla,lld,tmax):
    
    par = []
    Lt = []
    Lcpt = []
    nbMax = []
    compteur = 0
    
    for i in range(len(lla)):
        for j in range(len(lld)):
            par.append([lla[i],lld[j]])
            tmp_t,tmp_cpt = addProcess_2S(tmax, lla[i], [lld[j]], [0])
            Lcpt.append(dissocierList(tmp_cpt,0))
            Lt.append(tmp_t)
            nbMax.append(max(Lcpt[compteur]))
            compteur += 1;
            
    return par,nbMax

In [174]:
def g_ParaMax(param, lMax):
    
    a = np.argmax(lMax)
    
    return param[a]

In [150]:
def g_OverThreshold(param,nbMax,tH):
    
    lParam = []
    overTh = []
    
    for i in range(len(param)):
        if nbMax[i] >= tH:
            lParam.append(param[i])
            overTh.append(nbMax[i])
            
    
    return lParam,overTh

In [160]:
def g_TimeOverThreshold(lt,l_cpt,th):
    
    for i in range(len(lt)):
        if max(l_cpt[i]) >= th:
            return lt[i]
        
    return None

In [179]:
def test(param):
    
    res = []
    
    for i in param:
        res.append(i[0]-i[1])

    return res

In [210]:
def g_Average(L_cpt):
    
    somme = (len(L_cpt[0]))*[0.0]
    
    for i in range(len(L_cpt[0])):
        l = dissocierList(L_cpt,i)
        for j in l:
            somme[i] += j;
        somme[i] = somme[i]/len(l)
    
    return somme

In [212]:
def multiAddProcess():
    

In [213]:
#variables
lam_a = 10
lam_d = [20,20,20]
cpt = [0,0,0]
tmax = 300  # 300 pour 5 minutes de simulation

#Pour la collection de données
l_lama = [30,27,25,40,20]
l_lamd = [30,29,28,35,36]
th = 20  # treshold

In [217]:
param,nbMax = g_Max(l_lama,l_lamd,3600)
ltmp,lcpt = addProcess_2S(tmax,lam_a, lam_d, cpt)
moy_files = g_Average(lcpt)
paramMax = g_ParaMax(param,nbMax)
l_paramOT,l_nbMaxOT = g_OverThreshold(param,nbMax,th)
time_OT = g_TimeOverThreshold  # temps a partir du quel au moin une file a depassée la valeur th
ecart_lambda = test(param)  # list des ecart entre lambda dep et lambda arr (0 -> lamda egaux, >0 -> lambda_a > lambda_b, <0 -> inverse)

In [218]:
moy_files

[0.8653846153846154, 0.8461538461538461, 0.7115384615384616]